<a href="https://colab.research.google.com/github/scaairesearch/da_cv_fer/blob/main/evaluating_synthetic_data_2503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

25_03 model

In [1]:
#@title loading libraries
!git clone https://github.com/scaairesearch/da_cv_fer.git
import os
os.getcwd()
os.chdir('da_cv_fer')

Cloning into 'da_cv_fer'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 376 (delta 89), reused 158 (delta 81), pack-reused 209
Receiving objects: 100% (376/376), 26.51 MiB | 25.16 MiB/s, done.
Resolving deltas: 100% (203/203), done.


In [2]:
!pip install -q -r requirements.txt # -q is for quiet install
# for some reason the below does not get installed with requirements
!pip install -q opendatasets
!pip install -q facenet_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 746.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
#@title loading libraries
from da_cv_fer.data_config import DataConfig
from da_cv_fer.ds_sfew import DatasetSFEW
from da_cv_fer.ds_sfew_crop import DatasetSFEWCROP
from da_cv_fer.ds_expw_crop import EXPWCROP
from da_cv_fer.ds_expw import EXPW
from da_cv_fer.utils import *
from da_cv_fer.model_DANN import DANN
from da_cv_fer.model_DANN_Bespoke import DANNBespoke
from da_cv_fer.run_config import RunConfig
from da_cv_fer.train import *
import os
import zipfile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm import tqdm # for beautiful model training updates

import matplotlib.pyplot as plt # for plots

import time
from pathlib import Path

from copy import deepcopy
import numpy as np


In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


In [5]:

# Mounting Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
#@title Loading models to evaluate
non_dann_model_sfew_name =  'non_dann_sfew_expw_25_03.pt'
dataconfig_inference = DataConfig()

In [10]:
# NON DANN

non_dann_model_inference  = DANNBespoke(num_classes=7,backbone='mobilenet', attention=False, dropout = 0)
# model_dir = Path('/content/gdrive/MyDrive/CV_FER/models/non_dann_sfew_expw')
NON_DANN_SFEW_DIR = Path(dataconfig_inference.MODEL_DIR,"non_dann_sfew_expw") # Path(self.MODEL_DIR,"non_dann_sfew")

try:
    non_dann_model_inference.load_state_dict(torch.load(Path(NON_DANN_SFEW_DIR, non_dann_model_sfew_name),
                                                        map_location=torch.device(device)))
    print(f'{non_dann_model_sfew_name} locked and loaded')
except Exception as e:
    print(f' problem in loading {non_dann_model_sfew_name} {e}')

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 55.8MB/s]


feature extractor backbone created using mobilenet model
non_dann_sfew_expw_25_03.pt locked and loaded


In [6]:
#@title Get SFEW Dataset

sfew = DatasetSFEWCROP()
sfew_train_loader, sfew_val_loader = sfew.get_dataloader()
print(f' length of SFEW dataset : {len(sfew_train_loader.dataset)}')

 self.BASE_PATH -dataset, 
 self.EXTRACT_DIR-dataset/sfew,
 self.ZIP_FILE_PATH - dataset/SFEW_2.zip 
Directory dataset/sfew created successfully.
No files (including zip file) found in dataset/sfew.Copying file
Starting File copying from /content/gdrive/MyDrive/CV_FER/dataset/SFEW_2.zip to dataset/sfew
File copied successfully from /content/gdrive/MyDrive/CV_FER/dataset/SFEW_2.zip to dataset/sfew
Initiating extraction of dataset/sfew/SFEW_2.zip to dataset/sfew
...completed for dataset/sfew/SFEW_2.zip
dataset/sfew/Train Happy.zip Happy
...completed for dataset/sfew/Train/Happy.zip
dataset/sfew/Train Sad.zip Sad
...completed for dataset/sfew/Train/Sad.zip
dataset/sfew/Train Surprise.zip Surprise
...completed for dataset/sfew/Train/Surprise.zip
dataset/sfew/Train Fear.zip Fear
...completed for dataset/sfew/Train/Fear.zip
dataset/sfew/Train Angry.zip Angry
...completed for dataset/sfew/Train/Angry.zip
dataset/sfew/Train Neutral.zip Neutral
...completed for dataset/sfew/Train/Neutral.zip
da

Checking which images are misclassified in sfew train set

In [ ]:
  list_misclassified_images = get_misclassified_images_list(model= non_dann_model_inference,
                                                            device = device,
                                                            test_loader = sfew_train_loader,
                                                            num_image = 300)
  list_misclassified_images

In [ ]:
  (list_misclassified_images[0][1], list_misclassified_images[0][2],   list_misclassified_images[0][3])

(tensor([0., 0., 0., 0., 0., 0., 1.]),
 tensor([4]),
 'dataset/sfew/Train_Crop/Surprise/NanyDiaries_005401880_00000001.png')

In [ ]:
len(list_misclassified_images)

292

In [ ]:
import pandas as pd
df = pd.DataFrame(columns = ['image_name','ground_truth','predicted_class'])

labels_map={0:"Angry",
            1:"Disgust",
            2:"Fear",
            3:"Happy",
            4:"Sad",
            5:"Surprise",
            6:"Neutral"}
for i, entry in enumerate(list_misclassified_images):
  predicted_class = labels_map[torch.argmax(entry[1]).item()]
  ground_truth = labels_map[entry[2].item()]
  image_name = entry[3].split(os.sep)[-1]
  print(f'image {i} : {image_name} : {predicted_class} : {ground_truth}' )
  df.loc[i] = [image_name, ground_truth, predicted_class]
df

image 0 : NanyDiaries_005401880_00000001.png : Neutral : Sad
image 1 : HarryPotter_Half_Blood_Prince_010014374_00000046.png : Happy : Sad
image 2 : HarryPotter_Half_Blood_Prince_004418334_00000001.png : Fear : Sad
image 3 : SocialNetwork_002516407_00000027.png : Angry : Surprise
image 4 : SomethingBorrowed_011503440_00000040.png : Sad : Happy
image 5 : ItsComplicated_003539527_00000080.png : Happy : Disgust
image 6 : ACaseofYou_003118400_00000013.png : Neutral : Sad
image 7 : LittleManhattan_001740654_00000012.png : Neutral : Sad
image 8 : Bridesmaids_010123320_00000053.png : Surprise : Sad
image 9 : OneFlewOverCuckooNest_004651880_00000033.png : Surprise : Angry
image 10 : DecemberBoys_003642720_00000001.png : Angry : Sad
image 11 : MissMarch_000546640_00000065.png : Sad : Neutral
image 12 : SomethingBorrowed_012331480_00000001.png : Angry : Happy
image 13 : AlexEmma_003716360_00000040.png : Surprise : Sad
image 14 : SomethingBorrowed_001318800_00000047.png : Sad : Disgust
image 15 : 

,image_name,ground_truth,predicted_class
0,NanyDiaries_005401880_00000001.png,Sad,Neutral
1,HarryPotter_Half_Blood_Prince_010014374_000000...,Sad,Happy
2,HarryPotter_Half_Blood_Prince_004418334_000000...,Sad,Fear
3,SocialNetwork_002516407_00000027.png,Surprise,Angry
4,SomethingBorrowed_011503440_00000040.png,Happy,Sad
...,...,...,...
287,NanyDiaries_004118480_00000017.png,Fear,Neutral
288,HarryPotterAndOrderOPhoenix_015256413_00000011...,Fear,Angry
289,YouveGotAMail_000426174_00000017.png,Sad,Neutral
290,Juno_001934160_00000002.png,Disgust,Surprise


In [ ]:
df.to_excel("misclassified.xlsx")

In [11]:
non_dann_model_inference.eval()
all_preds = []
all_gts =[]

with torch.no_grad():
  for batch in sfew_val_loader:
    images, labels = batch[0].to(device), batch[1].to(device)
    class_output,_,_ = non_dann_model_inference(images)
    label_gts = labels.argmax(dim=1)
    label_preds = class_output.argmax(dim=1)
    all_gts.extend(label_gts.cpu().numpy())
    all_preds.extend(label_preds.cpu().numpy())

In [12]:
labels_map={0:"Angry",
            1:"Disgust",
            2:"Fear",
            3:"Happy",
            4:"Sad",
            5:"Surprise",
            6:"Neutral"}
all_preds = [labels_map[int_label] for int_label in all_preds]
all_gts= [labels_map[int_label] for int_label in all_gts]

# all_preds, all_gts

In [13]:
from sklearn.metrics import classification_report
# Calculate performance metrics
accuracy = (np.array(all_preds) == np.array(all_gts)).mean()
classification_rep = classification_report(all_gts, all_preds)

In [14]:
# Print the classification report
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy: 0.4606205250596659
Classification Report:
               precision    recall  f1-score   support

       Angry       0.57      0.48      0.52        75
     Disgust       0.18      0.18      0.18        22
        Fear       0.26      0.16      0.20        43
       Happy       0.65      0.83      0.73        72
     Neutral       0.30      0.40      0.34        52
         Sad       0.46      0.56      0.50        84
    Surprise       0.44      0.25      0.32        71

    accuracy                           0.46       419
   macro avg       0.41      0.41      0.40       419
weighted avg       0.45      0.46      0.45       419



In [15]:
all_gts = np.array(all_gts)
all_preds = np.array(all_preds)

# Get unique classes in the dataset
unique_classes = np.unique(all_gts)

# Initialize arrays to store class-wise correct predictions and total samples
class_correct = np.zeros(len(unique_classes))
class_total = np.zeros(len(unique_classes))

# Iterate over each unique class
for i, cls in enumerate(unique_classes):
    # Filter predictions and labels for the current class
    class_preds = all_preds[all_gts == cls]
    class_labels = all_gts[all_gts == cls]

    # Calculate accuracy for the current class
    class_correct[i] = np.sum(class_preds == class_labels)
    class_total[i] = len(class_labels)

# Print accuracy for each class
for i, cls in enumerate(unique_classes):
    if class_total[i] > 0:
        accuracy = 100 * class_correct[i] / class_total[i]
        print(f'Accuracy for class {cls}: {accuracy:.2f}% (Total Correct: {int(class_correct[i])}/{int(class_total[i])})')
    else:
        print(f'Accuracy for class {cls}: N/A (No samples in this class)')

Accuracy for class Angry: 48.00% (Total Correct: 36/75)
Accuracy for class Disgust: 18.18% (Total Correct: 4/22)
Accuracy for class Fear: 16.28% (Total Correct: 7/43)
Accuracy for class Happy: 83.33% (Total Correct: 60/72)
Accuracy for class Neutral: 40.38% (Total Correct: 21/52)
Accuracy for class Sad: 55.95% (Total Correct: 47/84)
Accuracy for class Surprise: 25.35% (Total Correct: 18/71)
